In [3]:
import tweepy
import pandas as pd
import time

# Authentication with Twitter API V2
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAADcsxQEAAAAA1FJXhrBL%2FXo3Nlw78YC5TVO4jns%3D30eM1mRWzb9erLFlvSrqupvdS9pByR6xE530Vn6gJmK26ychGb"

# Set up Tweepy client for API V2
client = tweepy.Client(bearer_token=BEARER_TOKEN)

# Define query groups (adjusted to fit under 512 characters)

query_group = '"Kosovo" AND "Serbia" OR ("terrorist" OR "NATO" OR "genocide" OR "occupier" OR "enemy" OR "traitor" OR "backward" OR "barbaric" OR "ethnic" OR "attack" OR "war crimes" OR "shqiptarë" OR "serbë" OR "pavarësi" OR "luftë" OR "Independence" OR "Albanian" OR "KosovoAlbanians" OR "Srbi" OR "Albanci" OR "nezavisnost" OR "rat" OR "SerbiaKosovo" OR "SerbNationalism" OR "SerbianPride" OR "KosovoSerbs" OR "United Nations" OR "peace agreement" OR "territory" OR "self-determination" OR "sovereignty" OR "border dispute" OR "international law" OR "KosovoLiberationArmy" OR "Yugoslavia" OR "Kosovo War" OR "1999" OR "SerbiaKosovoConflict" OR "military" OR "peace talks" OR "UN Resolution" OR "ethnic cleansing")'

query_groups = [
    # '"Kosovo" OR "Kosova" OR "Serbia" OR "Serb" OR "Albanians" OR "Serbs" OR "RKS" OR "SRB"',
    # '"#Kosovo" OR "#Serbia" OR "#KosovoSerbia" OR "#KosovoConflict" OR "#KosovoIndependence"',
    # '"terrorist" OR "occupier" OR "enemy" OR "traitor" OR "backward" OR "barbaric"',
    # '"genocide" OR "ethnic" OR "attack" OR "war crimes"',
    # '"Kosova" OR "shqiptarë" OR "serbë" OR "pavarësi" OR "luftë"',
    # '"Independence" OR "Albanian" OR "KosovoAlbanians" OR "Srbi" OR "Albanci"',
    # '"nezavisnost" OR "rat" OR "SerbiaKosovo" OR "SerbNationalism" OR "SerbianPride"',
    # '"KosovoSerbs" OR "NATO" OR "United Nations" OR "peace agreement"',
    # '"territory" OR "self-determination" OR "sovereignty" OR "border dispute"',
    # '"international law" OR "KosovoLiberationArmy" OR "Yugoslavia" OR "Kosovo War"',
    # '"1999" OR "SerbiaKosovoConflict" OR "military" OR "peace talks" OR "UN Resolution" OR "ethnic cleansing"'
     query_group
]

# Initialize variables
tweets_data = []
tweet_count = 0
next_token = None
tweet_ids = set()  # Set to track unique tweet IDs
max_tweets = 100  # Set the total number of tweets you want to retrieve per query group

# Function to save data to CSV after each request
def save_to_csv(data, filename="kosovo_serbia_tweetsunique.csv"):
    df = pd.DataFrame(data, columns=["Date", "User", "Tweet"])
    df.to_csv(filename, mode='a', header=not bool(tweet_count), index=False)
    print(f"Saved {len(data)} tweets to CSV.")

# Loop through each query group
for query in query_groups:
    group_tweet_count = 0  # Count tweets for each query group
    while group_tweet_count < max_tweets:
        try:
            if next_token:
                tweets = client.search_recent_tweets(query=query, max_results=100, next_token=next_token)
            else:
                tweets = client.search_recent_tweets(query=query, max_results=100)
            
            # Extract tweet details and store in list
            for tweet in tweets.data:
                if tweet.id not in tweet_ids:  # Check if tweet ID is already in the set
                    tweet_ids.add(tweet.id)  # Add the tweet ID to the set
                    tweets_data.append([tweet.created_at, tweet.author_id, tweet.text])
                    group_tweet_count += 1
                    tweet_count += 1  # Increment the overall tweet count
            
            # Save to CSV after fetching the group of tweets
            save_to_csv(tweets_data)
            
            # Update next_token for pagination to ensure no duplicate results
            next_token = tweets.meta.get('next_token') if 'next_token' in tweets.meta else None
            
            # Break if no more tweets are available in this query group
            if not next_token or group_tweet_count >= max_tweets:
                break

        except tweepy.TooManyRequests as e:
            # Wait for the rate limit window to reset
            print("Rate limit exceeded, waiting for 15 minutes...")
            time.sleep(15 * 60)  # Wait for 15 minutes

        except Exception as e:
            print(f"An error occurred: {e}")
            break

print(f"Total tweets collected: {tweet_count}")


Rate limit exceeded, waiting for 15 minutes...


KeyboardInterrupt: 